In [1]:
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import pandas as pd
import numpy as np

header_name = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'

In [25]:
def initial_538_predictions():
    """Creates dataframe with initial values from 538
    Returns:
        dataframe of 538 predictions
    """
    df_columns = ['date', 'team', 'win%']
    df = pd.DataFrame(columns=df_columns)
    
    output_file = 'z:\python_projects\aaa.exe'
    year = 2019
    result = ''
    URL = 'https://projects.fivethirtyeight.com/2019-nfl-predictions/games/?ex_cid=rrpromo'
    headers = {'User-Agent': header_name}
    source = requests.get(URL, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')

    # Finding year of predictions
    for timestamp in soup.findAll('div', attrs={'class': 'container'}):
        for year in timestamp.findAll('div', attrs={'id': 'intro'}):
            year = int(year.h1.get_text()[0:5])

    # Gathering data
    for week in soup.findAll('section', attrs={'class': 'week'}):
        if week.get_text()[5] == '6':
            for date in week.findAll('div', attrs={'class': 'days'}):
                for day in date.findAll('div', attrs={'class': 'day'}):  
                    for game in day.findAll('div', attrs={'class': 'game'}):

                        # Finding date of each game  
                        for h4 in day.findAll('h4', attrs={'class': 'h4'}):
                            date = h4.get_text().strip()
                            date = date.split(', ')[1]
                            date = date[0:3] + date[4:] + ' ' + str(year)
                            date = datetime.strptime(date, '%b %d %Y')
                            date = date.strftime('%m/%d/%Y')


                        # Finding data for each game
                        for game_body in game.findAll('table', attrs={'class': 'game-body'}):           
                            for num_teams, matchup in enumerate(game_body.findAll('tr', attrs={'class': 'tr'})):
                                squad = matchup.find('td', attrs={'class': 'td text team'})
                                win_percentage = matchup.find('td', attrs={'class': 'td number chance'}).get_text().strip()

                                if squad:
                                    team = squad.get_text().strip()
                                    result = ''
                                    
                                    if num_teams == 0:
                                        df = df.append(pd.Series([date, team, win_percentage], 
                                                                  index=df.columns), ignore_index=True)
                                    else:
                                        df = df.append(pd.Series(['', team, win_percentage], 
                                                                  index=df.columns), ignore_index=True)
    df.drop_duplicates(inplace=True)
    #datetime.strptime('Oct', '%b')
    #df.set_index('date')
    
    return df, year
    
def getting_spreadsheet(file_name):
    """Loads spreadsheet
    Args:
        file_name: path of file
    Returns:
        dataframe of data
    """
    return(pd.read_csv(file_name))
def predictions_538(week_num):
    """Loads 538 predictions
    Args:
        week_num: week number of games
    Returns:
        df: dataframe with 538 predictions
        year: year of games. Needed for Vegas odds
    """
    year = 2019
    result = ''
    URL = 'https://projects.fivethirtyeight.com/2019-nfl-predictions/games/?ex_cid=rrpromo'
    headers = {'User-Agent': header_name}
    source = requests.get(URL, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')
    cols = ['date', 'team', 'win%']
    blank_rows = ['']*(len(df.columns)-3)
    cols.extend(blank_rows)
    predictions = pd.DataFrame(columns=cols)

    # Finding year of predictions
    for timestamp in soup.findAll('div', attrs={'class': 'container'}):
        for year in timestamp.findAll('div', attrs={'id': 'intro'}):
            year = int(year.h1.get_text()[0:5])

    # Gathering data
    for week in soup.findAll('section', attrs={'class': 'week'}):
        if week.get_text()[5] == str(week_num):
            for date in week.findAll('div', attrs={'class': 'days'}):
                for day in date.findAll('div', attrs={'class': 'day'}):  
                    for game in day.findAll('div', attrs={'class': 'game'}):

                        # Finding date of each game  
                        for h4 in day.findAll('h4', attrs={'class': 'h4'}):
                            date = h4.get_text().strip()
                            date = date.split(', ')[1]
                            date = date[0:3] + date[4:] + ' ' + str(year)
                            date = datetime.strptime(date, '%b %d %Y')
                            date = date.strftime('%m/%d/%Y')


                        # Finding data for each game
                        for game_body in game.findAll('table', attrs={'class': 'game-body'}):           
                            for num_teams, matchup in enumerate(game_body.findAll('tr', attrs={'class': 'tr'})):
                                squad = matchup.find('td', attrs={'class': 'td text team'})
                                win_percentage = matchup.find('td', attrs={'class': 'td number chance'}).get_text().strip()

                                if squad:
                                    team = squad.get_text().strip()
                                    result = ''
                                    
                                    if num_teams == 0:
                                        row_data = [date, team, win_percentage]
                                        row_data.extend(blank_rows)
                                        
                                        predictions = predictions.append(pd.Series(row_data, index=predictions.columns), 
                                                                         ignore_index=True)
                                    else:
                                        row_data = ['', team, win_percentage]
                                        row_data.extend(blank_rows)

                                        predictions = predictions.append(pd.Series(row_data, index=predictions.columns), 
                                                                         ignore_index=True)
    #df.drop_duplicates(inplace=True)
    #datetime.strptime('Oct', '%b')
    #df.set_index('date')
    
    return predictions, year
def loading_odds(df, year):
    """Loads odds to spreadsheet
    Args:
        df: dataframe that odds will be written to
        year: year that will be used to gather the dates of the games
    Returns:
        dataframe of datawith odds
    """
    names = ['Open', 'odds','Westgate','MGM Mirage', 'betMGM',
             'William Hill', 'CG Technology', 'Circa Sports','Stations']
    book = pd.DataFrame(columns=names)
    output_file = 'z:\python_projects\aaa.exe'
    temp_away_list = []
    temp_home_list = []
    teams_list = []
    dates_list = []
    URL = 'http://www.vegasinsider.com/nfl/odds/las-vegas/money/'
    headers = {'User-Agent': header_name}
    source = requests.get(URL, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')

    for gameboard in soup.findAll('table', attrs={'class': 'viBodyContainerTble'}):
        for num, games in enumerate(gameboard.findAll('td', attrs={'class': 'viBodyBorderNorm'})):
            for game_info in games.findAll('td'):  
                for date in game_info.findAll('span', attrs={'class': 'cellTextHot'}):
                    date = date.get_text().split()[0]
                    month = date.split('/')[0]
                    day = date.split('/')[1]

                    if(int(day[0]) == 0):
                        day = day[1:]

                    date = str(month) + '/' + str(day) + '/' + str(year)
                    dates_list.extend([date, date])

                for team_name in game_info.findAll('a', attrs={'class': 'tabletext'}):
                    teams_list.append(team_name.get_text())

            for num, spread in enumerate(games.findAll('td',attrs={'class': 
                                                                  ['viCellBg1 cellTextNorm cellBorderL1 center_text nowrap',
                                                                   'viCellBg1 cellTextHot cellBorderL1 center_text nowrap',
                                                                   'viCellBg2 cellTextNorm cellBorderL1 center_text nowrap',
                                                                   'viCellBg2 cellTextHot cellBorderL1 center_text nowrap']})):           
                spread_text = spread.get_text().strip()
                if spread_text == '' or spread_text == 'XXXX':
                    away_spread = np.nan
                    home_spread = np.nan
                elif(spread_text[4] == '+' or spread_text[4] == '-'):
                    away_spread = spread_text[0:4]
                    home_spread = spread_text[4:]
                else:
                    away_spread = spread_text[0:5]
                    home_spread = spread_text[5:]

                temp_away_list.append(away_spread)
                temp_home_list.append(home_spread)
                
                if len(temp_home_list) == 9:
                    book = book.append(pd.Series(temp_away_list, index=names), ignore_index=True)
                    book = book.append(pd.Series(temp_home_list, index=names), ignore_index=True)
                    temp_away_list = []
                    temp_home_list = []

    book['team'] = teams_list
    book['date'] = dates_list

    book = book[book.date <= df.date.max()]    # Only getting odds for games in current week
    odds = book[['team', 'odds']]
    df = pd.merge(odds, df)[['date', 'team', 'win%', 'odds']]

    return df
def combining_data(predictions, df):
    """Appends current predictions to season long dataframe
    Args:
        predictions: dataframe of predictions
        df: dataframe of data
    Returns:
        df: combined dataframe
    """

    cols = df.columns
    df = pd.concat([df, predictions], ignore_index=True, )
    df = df[cols]
    df.replace(np.nan, '', inplace=True)
    return(df)
def game_outcomes(df, week_num):
    """Finds winners and losers of games
    Args:
        df: dataframe of data
        week_num: week of games
    Returns:
        winners, losers: lists of winners and loser of games
    """
    year = 2019
    result = ''
    URL = 'https://projects.fivethirtyeight.com/2019-nfl-predictions/games/?ex_cid=rrpromo'
    headers = {'User-Agent': header_name}
    source = requests.get(URL, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')
    winners = []
    losers = []

    # Finding year of predictions
    for timestamp in soup.findAll('div', attrs={'class': 'container'}):
        for year in timestamp.findAll('div', attrs={'id': 'intro'}):
            year = int(year.h1.get_text()[0:5])

    # Gathering data
    for week in soup.findAll('section', attrs={'class': 'week'}):
        if week.get_text()[5] == str(week_num):
            for date in week.findAll('div', attrs={'class': 'days'}):
                for day in date.findAll('div', attrs={'class': 'day'}):  
                    for game in day.findAll('div', attrs={'class': 'game'}):

                        # Finding date of each game  
                        for h4 in day.findAll('h4', attrs={'class': 'h4'}):
                            date = h4.get_text()
                            date = date.split(', ')[1]
                            date = date[0:3] + date[4:6] + ' ' + str(year)
                            date = datetime.strptime(date, '%b %d %Y')
                            date = date.strftime('%m/%d/%Y')


                        # Finding data for each game
                        for game_body in game.findAll('table', attrs={'class': 'game-body'}):           
                            for num_teams, matchup in enumerate(game_body.findAll('tr', attrs={'class': 'tr'})):
                                winner = matchup.find('td', attrs={'class': 'td text team winner'})
                                loser = matchup.find('td', attrs={'class': 'td text team loser'})

                                if winner:
                                    winners.append(winner.get_text().strip())
                                elif loser:
                                    losers.append(loser.get_text().strip())
                                else:
                                    continue
                                
    
    return winners, losers
def odds_checker(row):
    """Checks if odds for game are present
    Args:
        row: row of data
    Returns:
        
    """
    if row['odds'] != '':
        return(row['odds'])
    else:
        return(row['odds new'])  
def implied_probability(row):
    """Uses odds to determine implied probability
    Args:
        row: row of data from dataframe
    Returns:
        implied win probability if it exists
    """
    if 'implied' in row.index:
        return row['implied']
    elif row['odds'] == '':
        return('')
    elif row['odds'][0] == '+':
        return(round(100.0/(100+int(row['odds'][1:])), 2))
    elif row['odds'][0] == '-':
        return(round(int(row['odds'][1:])/(100.0+int(row['odds'][1:])), 2))
    else:
        return('')    
    
    
def pick(row):
    """Uses win% and odds to determine what team to pick
    Args:
        row: row of data from dataframe
    Returns:
        pick if there is one
    """
    if 'pick' in row.index:
        return(row['pick'])
    elif row['implied'] == '':
        return('')
    elif (float(row['win%'][:-1])/100.0 > row['implied']):
        return(row.team)
    else:
        return('')
    
def spreadsheet_formatter(df):
    """Formats spreadsheet
    Args:
        df: dataframe of data

    """
    if 'Unnamed: 0' in df.columns:
        df.drop('Unnamed: 0', axis=1, inplace=True)
        
    df.replace(np.nan, '', inplace=True)
    
def odds_formatter(row):
    """Formats odds
    Args:
        row: row of data from dataframe
    Returns:
        formatted odds
    """
    if int(row['odds']) >= 100:
        return('+' + str(row['odds']))
    if int(row['odds']) <= -100:
        return(str(row['odds']))
    else:
        return(str(row['odds'])) 
    
def team_won_lost(row, winners, losers):
    """Determines team outcomes
    Args:
        row: row of data from dataframe
    Returns:
        updated entry for winner/loser
    """
    if row['team'] in winners:
        return('w')
    elif row['team'] in losers:
        return('l')
    else:
        return('')
     
def money_won_lost(row):
    """Determines amount won lost
    Args:
        row: row of data from dataframe
    Returns:
        amount won/lost
    """
    if row['pick'] == '':
        return(0)
    else:
        if row['result'] == 'w':
            if row['odds'][0] == '+':
                return(int(row['odds'][1:]))
            else:
                return(100)
        elif row['result'] == 'l':
            if row['odds'][0] == '+':
                return(-100)
            else:
                return(int(row['odds']))
        else:
            return(0)
def writing_spreadsheet(df, filename):
    """Writing to spreadsheet
    Args:
        df: data
        file_name: path of file
    """
    df.to_csv(filename, index=False)

In [30]:
file_name = 'z:\\python projects\\week_5.csv'

df = getting_spreadsheet(file_name) # Retrieving Spreadsheet
spreadsheet_formatter(df)
df['odds'] = df.apply(lambda row: odds_formatter(row), axis=1)
winners, losers = game_outcomes(df, week_num=5)
df['result'] = df.apply(lambda row: team_won_lost(row, winners, losers), axis=1)
df['w/l'] = df.apply(lambda row: money_won_lost(row), axis=1)
df['total'] = df['w/l'].sum()
predictions, year = predictions_538(week_num=6)
predictions = loading_odds(predictions, year)
predictions['implied'] = predictions.apply(lambda row: implied_probability(row), axis=1)
predictions['pick'] = predictions.apply(lambda row: pick(row), axis=1)
df = combining_data(predictions, df)

writing_spreadsheet(df,  'z:\\python projects\\NFL Game Outcome Spreadsheet.csv')

In [ ]:
file_name = 'z:\\python projects\\week 5.csv'

df, year = initial_538_predictions()
df = loading_odds(df, year)
df['implied'] = df.apply(lambda row: implied_probability(row), axis=1)
df['pick'] = df.apply(lambda row: pick(row), axis=1)
winners, losers = game_outcomes(df)
df['result'] = df.apply(lambda row: team_won_lost(row, winners, losers), axis=1)
df['w/l'] = df.apply(lambda row: money_won_lost(row), axis=1)
df['total'] = df['w/l'].sum()
writing_spreadsheet(df, file_name)
df

In [ ]:
df = predictions_538(df)
df['odds'] = df.apply(lambda row: odds_formatter(row), axis=1)
df.replace(np.nan, '', inplace=True)
df['implied'] = df.apply(lambda row: implied_probability(row), axis=1)
df['pick'] = df.apply(lambda row: pick(row), axis=1)
winners, losers = game_outcomes(df)
df['result'] = df.apply(lambda row: team_won_lost(row, winners, losers), axis=1)
df['w/l'] = df.apply(lambda row: money_won_lost(row), axis=1)
df['total'] = df['w/l'].sum()
writing_spreadsheet(df, file_name)